# Lifestyle Products Recommender
In this notebook, I will use a transfer learning technique called ResNet50 to classify the images.

In [1]:
import tensorflow

In [3]:
# Importing the dependencies
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

In [4]:
# Initializing the ResNet50 model, we will not train the model as it is already trained.
model = ResNet50(weights = "imagenet", include_top = False, input_shape = (224,224,3))
model.trainable = False

In [5]:
# Model Building
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

In [6]:
# Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [7]:
# Feature Extraction
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

In [9]:
filenames = []

for file in os.listdir('images'):
    filenames.append(os.path.join('images',file))

In [11]:
feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))

100%|█████████████████████████████████████████████████████████████████████████| 44441/44441 [17:39:01<00:00,  1.43s/it]


In [12]:
# Saving the model to reuse later
pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))